# Capstone Project: Bringing It All Together

In this lab, you will bring together many of the tools and techniques that you have learned throughout this course into a final project. You can choose from many different paths to get to the solution. You could use AWS Managed Services, such as Amazon Comprehend, or use the Amazon SageMaker models. Have fun on whichever path you choose.

### Business scenario

You work for a training organization that recently developed an introductory course about machine learning (ML). The course includes more than 40 videos that cover a broad range of ML topics. You have been asked to create an application that will students can use to quickly locate and view video content by searching for topics and key phrases.

You have downloaded all of the videos to an Amazon Simple Storage Service (Amazon S3) bucket. Your assignment is to produce a dashboard that meets your supervisor’s requirements.

To assist you, all of the previous labs have been provided in this workspace.

## Lab steps

To complete this lab, you will follow these steps:

1. [Viewing the video files](#1.-Viewing-the-video-files)
2. [Transcribing the videos](#2.-Transcribing-the-videos)
3. [Normalizing the text](#3.-Normalizing-the-text)
4. [Extracting key phrases and topics](#4.-Extracting-key-phrases-and-topics)
5. [Creating the dashboard](#5.-Creating-the-dashboard)

## Submitting your work

1. In the lab console, choose **Submit** to record your progress and when prompted, choose **Yes**.

1. If the results don't display after a couple of minutes, return to the top of these instructions and choose **Grades**.

     **Tip**: You can submit your work multiple times. After you change your work, choose **Submit** again. Your last submission is what will be recorded for this lab.

1. To find detailed feedback on your work, choose **Details** followed by **View Submission Report**.

## Useful information

The following cell contains some information that might be useful as you complete this project.

In [ ]:
bucket = 'c46255a638438l1748394t1w538120888142-labbucket-12figcw8iu648'
job_data_access_role = 'arn:aws:iam::538120888142:role/service-role/c46255a638438l1748394t1w5-ComprehendDataAccessRole-1A1092NM0Q4C7'


## 1. Viewing the video files
([Go to top](#Challenge-Lab-8:-Bringing-It-All-Together))


The source video files are located in the following shared Amazon Simple Storage Service (Amazon S3) bucket.

In [ ]:
!aws s3 ls s3://aws-tc-largeobjects/CUR-TF-200-ACMNLP-1/video/

## 2. Transcribing the videos
([Go to top](#Challenge-Lab-8:-Bringing-It-All-Together))

Use this section to implement your solution to transcribe the videos.

In [ ]:
!aws s3 cp s3://aws-tc-largeobjects/CUR-TF-200-ACMNLP-1/video/ s3://{bucket}/input/ --recursive

In [ ]:
from boto3 import client

conn = client('s3') 
for key in conn.list_objects(Bucket=bucket)['Contents']:
    print(key['Key'])

In [ ]:
import boto3
import os, io, struct, json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import uuid
from time import sleep
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
transcribe_client = boto3.client("transcribe")

In [ ]:
output_files=[]
transcribe_output_prefix = 'transcribed'
for key in conn.list_objects_v2(Bucket=bucket, Prefix='input')['Contents']:
    if 'temp' in key['Key']:
        continue
    object_name=key['Key']
    media_input_uri = f's3://{bucket}/{object_name}'

    #create the transcription job
    job_uuid = uuid.uuid1()
    transcribe_job_name = f"transcribe-job-{job_uuid}"
    output_file = object_name.split('.')[0].replace(" ","_")
    transcribe_output_filename = f'{transcribe_output_prefix}-{output_file}.txt'
    output_files.append([transcribe_output_filename,object_name,""])
    print(f'{media_input_uri} transcribed to {transcribe_output_filename}')

    response = transcribe_client.start_transcription_job(
        TranscriptionJobName=transcribe_job_name,
        Media={'MediaFileUri': media_input_uri},
        MediaFormat='mp4',
        LanguageCode='en-US',
        OutputBucketName=bucket,
        OutputKey=transcribe_output_filename
    )

In [ ]:
print(output_files)

In [ ]:
job=None
while True:
    job = transcribe_client.get_transcription_job(TranscriptionJobName = transcribe_job_name)
    if job['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED','FAILED']:
        break
    print('.', end='')
    sleep(20)
        
print(job['TranscriptionJob']['TranscriptionJobStatus'])


In [ ]:
s3_client = boto3.client('s3')
transcribed_text = []
for transcribe_output_filename in output_files:
    result = s3_client.get_object(Bucket=bucket, Key=transcribe_output_filename[0]) 
    data = json.load(result['Body']) 
    transcription = data['results']['transcripts'][0]['transcript']
    transcribe_output_filename[2] = transcription

print(output_files[0])


## 3. Normalizing the text
([Go to top](#Challenge-Lab-8:-Bringing-It-All-Together))

Use this section to perform any text normalization steps that are necessary for your solution.

In [ ]:
import pandas as pd
df = pd.DataFrame(data=output_files, columns=['OutputFile','Video','Transcription'] )

In [ ]:
df.head()

In [ ]:

def normalize_text(content):
    text = re.sub(r"http\S+", "", content ) # Remove urls
    text = text.lower() # Lowercase 
    text = text.strip() # Remove leading/trailing whitespace
    text = re.sub('\s+', ' ', text) # Remove extra space and tabs
    text = re.sub('\n',' ',text) # remove newlines
    text = re.compile('<.*?>').sub('', text) # Remove HTML tags/markups:
    return text

In [ ]:
%%time
df['Transcription_normalized'] = df['Transcription'].apply(normalize_text)

In [ ]:
pd.set_option('display.max_colwidth', 150)
df.head()

## 4. Extracting key phrases and topics
([Go to top](#Challenge-Lab-8:-Bringing-It-All-Together))

Use this section to extract the key phrases and topics from the videos.

In [ ]:
s3_resource = boto3.Session().resource('s3')

def upload_comprehend_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

    

In [ ]:
comprehend_file = 'comprehend_input.csv'
prefix='capstone'
upload_comprehend_s3_csv(comprehend_file, 'comprehend', df['Transcription_normalized'].str.slice(0,5000))
test_url = f's3://{bucket}/{prefix}/comprehend/{comprehend_file}'
print(f'Uploaded input to {test_url}')

In [ ]:
# Comprehend client information
comprehend_client = boto3.client(service_name="comprehend")

# Other job parameters
input_data_format = 'ONE_DOC_PER_LINE'
job_uuid = uuid.uuid1()
job_name = f"kpe-job-{job_uuid}"
input_data_s3_path = test_url
output_data_s3_path = f's3://{bucket}/'

In [ ]:
# Begin the inference job
kpe_response = comprehend_client.start_key_phrases_detection_job(
    InputDataConfig={'S3Uri': input_data_s3_path,
                     'InputFormat': input_data_format},
    OutputDataConfig={'S3Uri': output_data_s3_path},
    DataAccessRoleArn=job_data_access_role,
    JobName=job_name,
    LanguageCode='en'
)

# Get the job ID
kpe_job_id = kpe_response['JobId']

In [ ]:
job_name = f'entity-job-{job_uuid}'
entity_response = comprehend_client.start_entities_detection_job(
    InputDataConfig={'S3Uri': input_data_s3_path,
                     'InputFormat': input_data_format},
    OutputDataConfig={'S3Uri': output_data_s3_path},
    DataAccessRoleArn=job_data_access_role,
    JobName=job_name,
    LanguageCode='en'
)
# Get the job ID
entity_job_id = entity_response['JobId']

## 5. Creating the dashboard
([Go to top](#Challenge-Lab-8:-Bringing-It-All-Together))

Use this section to create the dashboard for your solution.

Use the link below to obtain the IP address of your computer. 

http://checkip.amazonaws.com/

Copy the value displayed from the link above and replace the ip address below

In [ ]:
my_ip = "72.21.198.0/24"

In [ ]:
!pip install elasticsearch
!pip install requests
!pip install requests-aws4auth

Create an boto3 client for elasticsearch.

In [ ]:
es_client = boto3.client('es')

The following sets up an access policy so that only your ip address can access the elasticsearch dashboards.

In [ ]:
access_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "",
                "Effect": "Allow",
                "Principal": {
                    "AWS": "*"
                },
                "Action": "es:*",
                "Resource": "*",
                "Condition": { "IpAddress": { "aws:SourceIp": my_ip } }
            }
        ]
    }

Create the elasticsearch cluster using the following options:


- **DomainName** - is the name of the elasticsearch cluster
- **ElasticSearchClusterConfig** - specifies the instance type, the number of instances, whether a dedicated master is required, and if the cluster should be multi-zoned
- **AccessPolicies** - contains the statement from above that restricts access to only your IP address


In [ ]:
response = es_client.create_elasticsearch_domain(
    DomainName = 'nlp-lab',
    ElasticsearchVersion = '7.9',
    ElasticsearchClusterConfig={
        "InstanceType": 'm3.large.elasticsearch',
        "InstanceCount": 2,
        "DedicatedMasterEnabled": False,
        "ZoneAwarenessEnabled": False
    },
    AccessPolicies = json.dumps(access_policy)
)

Elasticsearch typically takes around 10 minutes to complete. Check the [Amazon ElasticSearch Service](https://console.aws.amazon.com/es/home?region=us-east-1#)

In [ ]:
# Get current job status
kpe_job = comprehend_client.describe_key_phrases_detection_job(JobId=kpe_job_id)

# Loop until job is completed
waited = 0
timeout_minutes = 30
while kpe_job['KeyPhrasesDetectionJobProperties']['JobStatus'] != 'COMPLETED':
    sleep(10)
    waited += 10
    assert waited//60 < timeout_minutes, "Job timed out after %d seconds." % waited
    print('.', end='')
    kpe_job = comprehend_client.describe_key_phrases_detection_job(JobId=kpe_job_id)

print('Ready')

In [ ]:
# Get current job status
entity_job = comprehend_client.describe_entities_detection_job(JobId=entity_job_id)

# Loop until job is completed
waited = 0
timeout_minutes = 30
while entity_job['EntitiesDetectionJobProperties']['JobStatus'] != 'COMPLETED':
    sleep(10)
    waited += 10
    assert waited//60 < timeout_minutes, "Job timed out after %d seconds." % waited
    print('.', end='')
    entity_job = comprehend_client.describe_entities_detection_job(JobId=entity_job_id)

print('Ready')

Once the results for both cells say 'Ready' you can proceed.

Get the output for the Key Phrases detection Job by extracting the output location from the job and downloading it to the file system.


In [ ]:
kpe_comprehend_output_file = kpe_job['KeyPhrasesDetectionJobProperties']['OutputDataConfig']['S3Uri']
print(f'output filename: {kpe_comprehend_output_file}')

kpe_comprehend_bucket, kpe_comprehend_key = kpe_comprehend_output_file.replace("s3://", "").split("/", 1)

s3r = boto3.resource('s3')
s3r.meta.client.download_file(kpe_comprehend_bucket, kpe_comprehend_key, 'output-kpe.tar.gz')

Next, extract the file and rename the output so we know which file this is.

In [ ]:
# Extract the tar file
import tarfile
tf = tarfile.open('output-kpe.tar.gz')
tf.extractall()
# Rename the output
!mv 'output' 'kpe_output'

You can repeat the above process for the entity detection job.

In [ ]:
entity_comprehend_output_file = entity_job['EntitiesDetectionJobProperties']['OutputDataConfig']['S3Uri']
print(f'output filename: {entity_comprehend_output_file}')

entity_comprehend_bucket, entity_comprehend_key = entity_comprehend_output_file.replace("s3://", "").split("/", 1)

s3r = boto3.resource('s3')
s3r.meta.client.download_file(entity_comprehend_bucket, entity_comprehend_key, 'output-entity.tar.gz')

# Extract the tar file
import tarfile
tf = tarfile.open('output-entity.tar.gz')
tf.extractall()
# Rename the output
!mv 'output' 'entity_output'

Read in the data from the Key Phrases file into an array.

In [ ]:
import json
data = []
with open ('kpe_output', "r") as myfile:
    for line in myfile:
        data.append(json.loads(line))


Load the data array into a dataframe. There are two columns, KeyPhrases and Line.

In [ ]:
kpdf = pd.DataFrame(data, columns=['KeyPhrases','Line'])
kpdf.head()

You can repeat the last 2 steps for the entities data.

In [ ]:
import json
data = []
with open ('entity_output', "r") as myfile:
    for line in myfile:
        data.append(json.loads(line))


In [ ]:
entitydf = pd.DataFrame(data, columns=['Entities','Line'])
entitydf.head()

Looking at the entities. the different detected entities are burried in the same fields. Depending on your scenario, you may want to split this out into separate columns for each entity type. To do this we can write a function.

In [ ]:
def extract_entities(entities, entity_type):
    filtered_entities=[]
    for entity in entities:
        if entity['Type'] == entity_type:
            filtered_entities.append(entity)
    return filtered_entities

Then we can apply the function to each of the event types we want to extract.

In [ ]:
        
# df['plot_normalized'] = df['plot'].apply(normalize_text)    
entitydf['location'] = entitydf['Entities'].apply(lambda x: extract_entities(x, 'LOCATION'))
entitydf['organization'] = entitydf['Entities'].apply(lambda x: extract_entities(x, 'ORGANIZATION'))

entitydf.head()

With the results from Comprehend loaded into dataframes, it's time to merge everything together. The **Line** will merge together the results from Comprehend with the original dataframe.

Start by setting the index on both results dataframes to the **Line** column.

In [ ]:
entitydf.set_index('Line', inplace = True)
entitydf.sort_index(inplace=True)
kpdf.set_index('Line', inplace=True)
kpdf.sort_index(inplace=True)
entitydf.head()

Next, merge the **kpdf** dataframe with the **entitydf** dataframe.

In [ ]:
m1 = kpdf.merge(entitydf, left_index=True, right_index=True)
m1.sort_index(inplace=True)
pd.set_option('display.max_colwidth', 200)
m1.head()

Now merge the **m1** dataframe with the original dataframe **df**.

In [ ]:
mergedDf = df.merge(m1, left_index=True, right_index=True)

In [ ]:
mergedDf.head()

In [ ]:
pd.set_option('display.max_colwidth', 50)
mergedDf.head()

In [ ]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import requests


In [ ]:
from time import sleep
alive = es_client.describe_elasticsearch_domain(DomainName='nlp-lab')
while alive['DomainStatus']['Processing']:
    print('.', end='')
    sleep(10)
    alive = es_client.describe_elasticsearch_domain(DomainName='nlp-lab')
    
print('ready!')

In [ ]:
es_domain = es_client.describe_elasticsearch_domain(DomainName='nlp-lab')
es_endpoint = es_domain['DomainStatus']['Endpoint']

Create an elasticsearch client using the following:

In [ ]:
region= 'us-east-1' # us-east-1
service = 'es' # IMPORTANT: this is key difference while signing the request for proxy endpoint.
credentials = boto3.Session().get_credentials()

awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)
es = Elasticsearch(
    hosts = [{'host': es_endpoint, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [ ]:
transcription = mergedDf.iloc[3,2]
keyphrases = mergedDf.iloc[3,4]
location = mergedDf.iloc[3,6]
organization = mergedDf.iloc[3,7]
movie_name = mergedDf.iloc[3,1]

document = {"name": movie_name, "transcription": transcription, "keyphrases": keyphrases, "location":location, "organization": organization}
print(document)

In [ ]:
from elasticsearch import helpers

def gendata(start, stop):    
    if stop>mergedDf.shape[0]:
        stop = mergedDf.shape[0]
    for i in range(start, stop):
        yield {
            "_index":'movies',
            "_type": "_doc", 
            "_id":i, 
            "_source": {"name": mergedDf.iloc[i,1], "transcription": mergedDf.iloc[i,2], "keyphrases": mergedDf.iloc[i,4], "location":mergedDf.iloc[i,6], "organization": mergedDf.iloc[i,7]}
        }

Next, you need to get some up to date credentials for the elasticsearch service, then call **helpers.bulk** to upload the remaining documents. This should take around 1 minute.

In [ ]:
%%time
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)
es = Elasticsearch(
    hosts = [{'host': es_endpoint, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
helpers.bulk(es, gendata(0,mergedDf.shape[0]))

# Create the Kibana Dashboard

In this section, you will create a Kibana Dashboard to display and filter the results.

First, grab the url for the Kibana dashboard.

In [ ]:
print(f'https://{es_endpoint}/_plugin/kibana')

1. Navigate to the kibana URL printed from the previous cell.
1. Once the page loads, select **Dashboard**.
1. Since this is the first time the dashboard is loaded, an **Index Pattern** will need to be defined. Select `Create index pattern`. 
1. Enter **movie*** as the **index pattern name**. You should see that the index pattern matches 1 source.
1. Choose 'Next step'.
1. Choose `Create index pattern`.
1. You should see a table of fields displayed. If everything is working, you will see 28 fields.
1. Choose the hamburger menu, and select 'Discover' from the list.
1. In the available field list on the left, move to the **name** field and choose `add` when it appears.
1. Choose `Save`.
1. Enter **movies** as the title and choose `Save'.
1. Choose the hamburger menu, and select 'Dashboard' from the list.
1. Choose `Create new dashboard`.
1. Choose `Add`.
1. Select **Movies** from the list.
1. Close the **Add Panels** pane.
1. Choose `Create New`.
1. Select **Tag Cloud** from the list of Visualizations.
1. Choose **movie*** as the source.
1. Under **Buckets** select `Add`, then choose **Tags**.
1. Choose **Terms** as the `Aggregation`.
1. Choose **keyphrases.Text.keyword** as the field.
1. Enter **25** as the size.
1. Select `Update`.
1. Select `Save`.
1. Enter **Key Phrases** as the `Title`.
1. Choose `Save and return`

1. Repeat steps 16-26 for the following fields:
    - location.Text.keyword
    - organization.Text.keyword

1. Choose 'Create new'.
1. Select **Metric** from the list of Visualizations.
1. Choose **movie*** as the source.
1. Select `Save`
1. Enter **Total Documents** as the `Title`.
1. Choose `Save and return`

1. Select the calendar icon.
1. From the **Commonly used** list, select **Today**.
1. Select the calendar icon again and update the **Refresh every** to 5 seconds.
1. Choose `Start`.

1. Choose `Save`
1. Enter **Movies** as the title.
1. Choose `Save`

With the dashboard created, you can proceed to upload the remaining documents. There are some helper functions that allow you to do this quickly. First define a function that will create the document.

# Cleanup

Once you have finished experimenting with elasticsearch, you can shutdown the cluster using the following:


In [ ]:
response = es_client.delete_elasticsearch_domain(
    DomainName='nlp-lab'
)

Elasticsearch typically takes around 10 minutes to complete. While that is happening you can explore some other techniques.

# Congratulations!

You have completed this lab, and you can now end the lab by following the lab guide instructions.

*©2021 Amazon Web Services, Inc. or its affiliates. All rights reserved. This work may not be reproduced or redistributed, in whole or in part, without prior written permission from Amazon Web Services, Inc. Commercial copying, lending, or selling is prohibited. All trademarks are the property of their owners.*
